In [ ]:
import pandas as pd 
import folium
import osmnx as ox
from geopy import distance

### Load data

In [ ]:
meta_path = "./datasets/scats/metadata.csv"
data_path = "./datasets/scats/2023/july/data.csv"
meta_df = pd.read_csv(meta_path)
data_df = pd.read_csv(data_path)

pd.set_option('display.max_rows', 500)

### Data info

In [ ]:
meta_df.head()

In [ ]:
data_df.head()

In [ ]:
meta_df.info()

In [ ]:
data_df.info(show_counts=True)

### Cleaning


Now I need to keep only those sites in data df which are also present in meta, since I only know locations of those sites so any other are pretty much useless to me.
data_df only has those rows in which Site and Region is present in meta_df.

In [ ]:
meta_site_region_pairs = meta_df[['SiteID', 'Region']].drop_duplicates()

# check both Region and Site
filtered_data_df = data_df[data_df['Site'].isin(meta_site_region_pairs['SiteID']) & data_df['Region'].isin(meta_site_region_pairs['Region'])];
filtered_data_df.info(show_counts=True)

In [ ]:
data_df = filtered_data_df

Dropping unnecessary columns

In [ ]:
data_df.drop(columns=['Avg_Volume', 'Weighted_Avg', 'Weighted_Var', 'Weighted_Std_Dev'], errors='ignore', inplace=True)
data_df.head()

Now for the same 'Region', 'Site' pair, sum of the 'Sum_Volume' is the total volume for that site in that region.

In [ ]:
# group by 'Detector'
grouped_data_df = data_df.groupby(['End_Time', 'Region', 'Site']).agg({'Sum_Volume': 'sum'}).reset_index()
grouped_data_df.head()

In [ ]:
# sanity checks
# for CCITY and Site 1, is the sum 239 and End_Time 20230701000000
data_df[(data_df['Region'] == 'CCITY') & (data_df['Site'] == 1) & (data_df['End_Time'] == 20230701000000)].sum()

All looks good now, assign and rename some columns

In [ ]:
data_df = grouped_data_df
data_df.rename(columns={'End_Time': 'Time', 'Sum_Volume': 'Volume'}, inplace=True)
data_df.head()

In [ ]:
# time in format YYYYMMDDHHMMSS
data_df['Time'] = pd.to_datetime(data_df['Time'], format='%Y%m%d%H%M%S')
data_df


Add metadata to data df

In [ ]:
meta_df.rename(columns={'SiteID': 'Site'}, inplace=True)

# for all SiteID, Region pairs in data_df, add Lat, Long, Site_Type from meta_df
data_df = data_df.merge(meta_df[['Site', 'Region', 'Lat', 'Long', 'Site_Type']], on=['Site', 'Region'], how='left')
data_df.head()

In [ ]:
# for some sites there's no meta
data_df[data_df['Lat'].isna()]
# count where any of the columns is na or NaN
data_df.isna().sum()
# drop these roww
data_df.dropna(inplace=True)

The final ready df

In [ ]:
# how many unique site,reion pairs are there
print("Total Sites:\n", data_df[['Site', 'Region']].drop_duplicates().count())

In [ ]:
data_df.head()

# ?? Why is there a 2 site difference in next timestamp filter vs no of sites

`data_df` is the final dataframe which has all the necessary information

### Plotting markers for time 2023-07-01 00:00:00

In [ ]:
data_df_backup = data_df.copy()
data_df = data_df[data_df['Time']==pd.to_datetime('2023-07-01 00:00:00')]
data_df

Now plotting roads and the markers together

In [ ]:
place = "Dublin, Ireland"
custom_filters = '["highway"~"motorway|trunk|primary|secondary|tertiary|unclassified"]'
G = ox.graph_from_place(place, network_type="drive", simplify=True, custom_filter=custom_filters)

# simplify the network to reduce the number of nodes and edges
G_proj = ox.project_graph(G)
G = ox.consolidate_intersections(G_proj, tolerance=25, rebuild_graph=True, dead_ends=True)

# plot the network
gdf = ox.graph_to_gdfs(G, nodes=False)

In [ ]:
m = gdf.explore()

# add markers for all sites
for i in range(0, len(data_df)):
    folium.Marker([data_df.iloc[i]['Lat'], data_df.iloc[i]['Long']], popup=data_df.iloc[i]['Site'], icon_size=(0.1,0.1)).add_to(m)

folium.TileLayer('cartodbpositron').add_to(m)

m

### Generating an edge list

In [ ]:
# distances in metres between two lat, long points
def get_dist_lat_long(lat1, long1, lat2, long2):
    return distance.distance((lat1, long1), (lat2, long2)).m

In [ ]:
# compress coordinates for data_df['Site], reindexing from Site 0 to Site 608 for a total of 609 sites
old_site_column = data_df['Site'].copy()
data_df['Site'] = data_df['Site'].astype('category')
data_df['Site'] = data_df['Site'].cat.codes
new_site_column = data_df['Site'].copy()

mapping = dict(zip(old_site_column, new_site_column))
display(mapping)

In [ ]:
# create an adjancecy matrix for the sites
site_count = len(data_df['Site'].unique())
assert(site_count == 609 and site_count == len(data_df['Site']))

adj_matrix = [[0 for i in range(site_count)] for j in range(site_count)]

for i in range(0, site_count):
    for j in range(0, site_count):
        if i == j:
            continue
        adj_matrix[i][j] = get_dist_lat_long(data_df.iloc[i]['Lat'], data_df.iloc[i]['Long'], data_df.iloc[j]['Lat'], data_df.iloc[j]['Long'])

This is a hyperparameter for the model, I will use this to generate the edge list for the model

In [ ]:
cut_off_distance = 2000

In [ ]:
binary_edges = [[0 if adj_matrix[i][j] > cut_off_distance else 1 for j in range(site_count)] for i in range(site_count)]

In [ ]:
# count of edges
sum([sum(i) for i in binary_edges])

In [ ]:
# export both the adjacency matrix
pd.DataFrame(adj_matrix).to_csv("./datasets/scats/2023/july/adj_matrix.csv", index=False, header=False)


### Transforming the full df to same ids

In [ ]:
full_df = data_df_backup.copy()
full_df['Site'] = full_df['Site'].map(mapping)

# save the full data
full_df.to_csv("./datasets/scats/2023/july/processed.csv", index=False)